# Perform Necessary Imports

In [1]:
import pandas as pd
import xgboost
import re
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score


# Read filtered Dataset

In [3]:
totalData = pd.read_csv('../datasets/nela10.csv')

# How it looks like?

In [5]:
totalData

id        date  \
0      abcnews--2019-01-30--Who is Stacey Abrams the ...  2019-01-30   
1      abcnews--2019-01-30--Ban government shutdowns ...  2019-01-30   
2      abcnews--2019-01-30--Democrats renew push for ...  2019-01-30   
3      abcnews--2019-01-30--Trump attacks intel chief...  2019-01-30   
4      abcnews--2019-01-30--Border security battle be...  2019-01-30   
...                                                  ...         ...   
57152  zerohedge--2019-11-01--Hong Kong Officer Faces...  2019-11-01   
57153  zerohedge--2019-11-01--Enough "Quid Pro Quo" G...  2019-11-01   
57154  zerohedge--2019-11-01--"Born For This? I Don't...  2019-11-01   
57155  zerohedge--2019-11-01--How Iran Used Google To...  2019-11-01   
57156  zerohedge--2019-11-01--Malaysia Rejects Goldma...  2019-11-01   

          source                                              title  \
0        abcnews  Who is Stacey Abrams, the Democrat who will re...   
1        abcnews  Ban government shutdowns? Some Republicans and...   
2        abcnews  Democrats renew push for equal pay for equal work   
3        abcnews  Trump attacks intel chiefs after they contradi...   
4        abcnews  Border security battle begins as Congress grap...   
...          ...                                                ...   
57152  zerohedge  Hong Kong Officer Faces Death Threats After Fi...   
57153  zerohedge                 Enough "Quid Pro Quo" Gaslighting!   
57154  zerohedge  "Born For This? I Don't Think So" - Trump Mock...   
57155  zerohedge  How Iran Used Google To Disrupt 5% Of Global O...   
57156  zerohedge  Malaysia Rejects Goldman's Offer Of "Less Than...   

                                                 content  \
0      Last November, Stacey Abrams, a Democrat who r...   
1      As lawmakers scramble to negotiate a border se...   
2      Ten years after President Barack Obama signed ...   
3      President Donald Trump on Wednesday declared "...   
4      Republicans and Democrats on the exclusive com...   
...                                                  ...   
57152  It's unclear exactly when it happened (the BBG...   
57153  Horse trading is the oxygen of politics; it is...   
57154  Having plunged from over 10% to just 1% in the...   
57155  Officials at Saudi Aramco believe that Iran us...   
57156  In an interview with the Financial Times, Mala...   

                                   author  \
0                        Cheyenne Haslett   
1        Benjamin Siegel,\nTrish Turner\n   
2                          John Parkinson   
3                         Meridith Mcgraw   
4      John Parkinson,\nBenjamin Siegel\n   
...                                   ...   
57152                        Tyler Durden   
57153                        Tyler Durden   
57154                        Tyler Durden   
57155                        Tyler Durden   
57156                        Tyler Durden   

                                                     url  \
0      https://abcnews.go.com/Politics/stacey-abrams-...   
1      https://abcnews.go.com/Politics/ban-government...   
2      https://abcnews.go.com/Politics/dems-renew-pus...   
3      https://abcnews.go.com/Politics/trump-attacks-...   
4      https://abcnews.go.com/Politics/border-securit...   
...                                                  ...   
57152  http://feedproxy.google.com/~r/zerohedge/feed/...   
57153  http://feedproxy.google.com/~r/zerohedge/feed/...   
57154  http://feedproxy.google.com/~r/zerohedge/feed/...   
57155  http://feedproxy.google.com/~r/zerohedge/feed/...   
57156  http://feedproxy.google.com/~r/zerohedge/feed/...   

                             published  published_utc  collection_utc  \
0            2019-01-30 19:10:47+00:00     1548893447      1567550233   
1            2019-01-30 17:29:04+00:00     1548887344      1567550233   
2            2019-01-30 21:35:45+00:00     1548902145      1567550233   
3            2019-01-30 17:03:40+00:00     1548885820      15

# Keep only content, Reliability as columns

In [6]:
#Keep only content, Reliability as columns
totalData = totalData.drop(['id','date','source','title','author','url','published','published_utc','collection_utc'],axis=1)

In [7]:
totalData

content  Reliability
0      Last November, Stacey Abrams, a Democrat who r...            0
1      As lawmakers scramble to negotiate a border se...            0
2      Ten years after President Barack Obama signed ...            0
3      President Donald Trump on Wednesday declared "...            0
4      Republicans and Democrats on the exclusive com...            0
...                                                  ...          ...
57152  It's unclear exactly when it happened (the BBG...            2
57153  Horse trading is the oxygen of politics; it is...            2
57154  Having plunged from over 10% to just 1% in the...            2
57155  Officials at Saudi Aramco believe that Iran us...            2
57156  In an interview with the Financial Times, Mala...            2

[57157 rows x 2 columns]

In [8]:
#find out unique Reliability labels type
totalData.Reliability.unique()

array([0, 2])

In [11]:
#Drop off mixed reliability(label=1) type
totalData = totalData[totalData.Reliability != 1]

In [12]:
#find out unique Reliability labels type
totalData.Reliability.unique()

array([0, 2])

# Cleaning dataset

In [13]:
totalData = totalData.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
totalData.content = totalData.content.apply(clean_text)
totalData.content = totalData.content.str.replace('\d+', '')

In [14]:
totalData.content.sample(10)

34010    one afternoon in the summer of  bob gramling d...
25620    why isnt jason clarke more famous hes very goo...
14889    tender buttons has closed its doors and we hav...
53331    rahaf alqunun is not alonethe only thing more ...
30527    britain is bracing for the coldest night of th...
1137     singapore a pristine white statue of a man in ...
2944     three teenagers have been arrested after a yea...
4658     the partialgovernment shutdownhas glided into ...
18744    gordon sondland the us ambassador to the europ...
7751     four out of five people identified as suspects...
Name: content, dtype: object

# Perform test train split

In [15]:
# split the dataset into training and validation datasets 
from sklearn.model_selection import train_test_split
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['tweet'], train['label'])
train_x, valid_x, train_y, valid_y = train_test_split(totalData['content'], totalData['Reliability'], test_size=0.20)

# Develop Unigram and Bigram Tf-Idf feature vectors from data

In [16]:
# unigram level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=None)
tfidf_vect.fit(totalData['content'])
xtrain_tfidf_unigram =  tfidf_vect.transform(train_x)
xvalid_tfidf_unigram =  tfidf_vect.transform(valid_x)

# bigram level tf-idf (bigram in this case)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,2), max_features=None)
tfidf_vect_ngram.fit(totalData['content'])
xtrain_tfidf_bigram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_bigram =  tfidf_vect_ngram.transform(valid_x)


# Build Model functions

In [42]:
def train_model(classifier, feature_vector_train, label,  feature_vector_valid, valid_y,is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    #print("In Validation Data",metrics.accuracy_score(predictions, valid_y))
    print("f1 score: ",f1_score(valid_y,predictions,average='weighted'))        
    return metrics.accuracy_score(valid_y,predictions)
    

# Base Line Model Used
## 1. Naive Bayes
## 2. Linear Classifier
## 3. Bagging
## 4. Boosting
## 5. SVM

#  Naive Bayes Model 

In [43]:
# Naive Bayes on Word Level TF IDF Vectors
print("For Unigram Tf-IDF features vectors using Naive Bayes")
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram, valid_y)
print ("Accuracy: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
print("For Bigram Tf-IDF features vectors using Naive Bayes")
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram, valid_y)
print ("Accuracy: ", accuracy)

For Unigram Tf-IDF features vectors using Naive Bayes
f1 score:  0.6949164043381264
Accuracy:  0.7856892932120364
For Bigram Tf-IDF features vectors using Naive Bayes
f1 score:  0.707853317795271
Accuracy:  0.7913750874737578


# Linear Classifier

In [47]:
# Linear Classifier on Word Level TF IDF Vectors
print("For Unigram TF-Idf feature vectors using Logistic Regression")
accuracy = train_model(linear_model.LogisticRegression(max_iter=500), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram, valid_y)
print("accuracy: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
print("For Biigram TF-Idf feature vectors using Logistic Regression")
accuracy = train_model(linear_model.LogisticRegression(max_iter=500),  xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram, valid_y)
print("Accuracy: ", accuracy)

For Unigram TF-Idf feature vectors using Logistic Regression
f1 score:  0.8778954704561857
accuracy:  0.887071378586424
For Biigram TF-Idf feature vectors using Logistic Regression
f1 score:  0.8258199257551274
Accuracy:  0.85444366689993


# Bagging Model

In [48]:
# RF on Word Level TF IDF Vectors
print("For Unigram Tf-Idf feature vectors using Random Forest Classifier")
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram, valid_y)
print ("Accuracy: ", accuracy)

# RF on ngram Level TF IDF Vectors
print("For Bigram Tf-Idf feature vectors using Random Forest Classifier")
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram, valid_y)
print ("Accuracy: ", accuracy)

For Unigram Tf-Idf feature vectors using Random Forest Classifier
f1 score:  0.7889982224045017
Accuracy:  0.8326627011896431
For Bigram Tf-Idf feature vectors using Random Forest Classifier
f1 score:  0.819699814467292
Accuracy:  0.8511196641007698


# Boosting Model

In [49]:
print("For Unigram Tf-Idf feature vectors using Extreme Gradient Boosting")
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_unigram.tocsc(), train_y, xvalid_tfidf_unigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)

# Extereme Gradient Boosting on ngram Level TF IDF Vectors
print("For Bigram Tf-Idf feature vectors using Extreme Gradient Boosting")
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_bigram.tocsc(), train_y, xvalid_tfidf_bigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)

For Unigram Tf-Idf feature vectors using Extreme Gradient Boosting
f1 score:  0.8889110797087082
Accuracy:  0.8964310706787963
For Bigram Tf-Idf feature vectors using Extreme Gradient Boosting
f1 score:  0.8775083946497313
Accuracy:  0.8889958012596221


# SVM Model

In [50]:
print("For Unigram Tf-Idf feature vectors using SVM")
#SVM Model on Unigram TF-IDF
accuracy = train_model(svm.SVC(), xtrain_tfidf_unigram.tocsc(), train_y, xvalid_tfidf_unigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)

# SVM Model on Bigram TF-IDF
print("For Bigram Tf-Idf feature vectors using SVM")
accuracy = train_model(svm.SVC(), xtrain_tfidf_bigram.tocsc(), train_y, xvalid_tfidf_bigram.tocsc(), valid_y)
print("Accuracy: ", accuracy)

For Unigram Tf-Idf feature vectors using SVM
f1 score:  0.9001589355399741
Accuracy:  0.9065780265920224
For Bigram Tf-Idf feature vectors using SVM
f1 score:  0.84919268919277
Accuracy:  0.8704513645906228
